In [1]:
import numpy as np
import pickle as pk

import tensorflow.keras
from tensorflow.keras.models import load_model
    
test_data_path = "test_data.pk"
MODEL_NAME = "model.keras"

model = load_model(MODEL_NAME)

2024-01-02 15:51:59.929923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
test_data = pk.load(open(test_data_path, "rb"))
X_test = test_data["X"]
alphabet_size = test_data["alphabet_size"]

y_test = test_data["y"]
event_mapping = test_data["event_mapping"]

len(X_test),len(y_test)

(1732222, 1732222)

X_pred = list()
for idx, seq in enumerate(X_test):
    current_prediction = None
    for i in range(len(seq)):
        pred = model(np.array(seq[:i+1]))
        if current_prediction is None:
            current_prediction = pred
        else:
            current_prediction = np.vstack((current_prediction, pred))
    X_pred.append(current_prediction)
    break
    if idx % 1000 == 0:
        print(idx)
    if idx == 15000:
        break

In [3]:
import timeit
import copy
start_time = timeit.default_timer()

X_pred = list()
for idx, seq in enumerate(X_test):

    x_src = copy.copy(seq)
    x_src.insert(0, event_mapping["<SOS>"])
    
    current_prediction = model(np.array(x_src))

    #print(x_src)
    #for i in range(current_prediction.shape[0]):
    #    print(np.argmax(current_prediction[i]))
    #raise Exception("Stop")

    X_pred.append(current_prediction)
    if idx % 5000 == 0:
        print(idx)
    if idx == 15000: # TODO: do more here
        break

elapsed_time = timeit.default_timer() - start_time
print(elapsed_time)

0
5000
10000
15000
282.46046023799863


In [4]:
event_mapping["<SOS>"]

268

In [5]:
X_test[0], X_pred[0].shape

([13, 13, 13, 13, 13, 13, 55, 55, 13, 13], TensorShape([11, 321]))

In [6]:
#### import pickle as pk

pk.dump(X_pred, open("X_pred.pk", "wb"))